In [16]:
import os
import sys
import pickle

from cities.utils.cleaning_utils import find_repo_root

root = find_repo_root()


import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from cities.utils.cleaning_utils import standardize_and_scale

import matplotlib.pyplot as plt



from cities.utils.data_grabber import DataGrabber
data = DataGrabber()
data.get_features_wide(["gdp", "population", "transport",
            "spending_transportation", "spending_commerce", "spending_HHS"])
gdp = data.wide['gdp']
# population = data.wide['population']
# trnasport = data.wide['transport']
# spending_transportation = data.wide['spending_transportation']
# spending_commerce = data.wide['spending_commerce']

def tableInfo(tableName):

    print(tableName.head())
    print(tableName.dtypes)
    print(f'Number of rows: {tableName.shape[0]}')
    print(f'Unique FIPS numbers {tableName['GeoFIPS'].nunique()}')

In [25]:
column_names = ['col1', 'metro_area', 'col3', 'GeoName']

metro_areas = pd.read_csv(f"{root}/data/raw/metrolist.csv", header=None, names=column_names)



unique_metro_areas = metro_areas['metro_area'].nunique()
print(f"Number of unique metropolitan areas {unique_metro_areas}")

unique_geo_names = metro_areas['GeoName'].nunique()
print(f"Number of unique Counties {unique_geo_names}")

Number of unique metropolitan areas 384
Number of unique Counties 1160


In [18]:
metro_areas

,col1,metro_area,col3,GeoName
0,10180,"Abilene, TX (Metropolitan Statistical Area)",48059,"Callahan, TX"
1,10180,"Abilene, TX (Metropolitan Statistical Area)",48253,"Jones, TX"
2,10180,"Abilene, TX (Metropolitan Statistical Area)",48441,"Taylor, TX"
3,10420,"Akron, OH (Metropolitan Statistical Area)",39133,"Portage, OH"
4,10420,"Akron, OH (Metropolitan Statistical Area)",39153,"Summit, OH"
...,...,...,...,...
1155,49660,"Youngstown-Warren-Boardman, OH-PA (Metropolita...",39155,"Trumbull, OH"
1156,49660,"Youngstown-Warren-Boardman, OH-PA (Metropolita...",42085,"Mercer, PA"
1157,49700,"Yuba City, CA (Metropolitan Statistical Area)",6101,"Sutter, CA"
1158,49700,"Yuba City, CA (Metropolitan Statistical Area)",6115,"Yuba, CA"


In [38]:
# compare geonames with cities source of truthe



# GeoNames that are not common with GDP
geo_names_not_in_gdp = metro_areas[~metro_areas['GeoName'].isin(gdp['GeoName'])]
geo_names_not_in_gdp


,col1,metro_area,col3,GeoName
135,13980,"Blacksburg-Christiansburg, VA (Metropolitan St...",51933,"Montgomery + Radford, VA"
153,14500,"Boulder, CO (Metropolitan Statistical Area)",8013,"Boulder, CO"
210,16820,"Charlottesville, VA (Metropolitan Statistical ...",51901,"Albemarle + Charlottesville, VA"
327,19740,"Denver-Aurora-Lakewood, CO (Metropolitan Stati...",8001,"Adams, CO"
329,19740,"Denver-Aurora-Lakewood, CO (Metropolitan Stati...",8014,"Broomfield, CO"
335,19740,"Denver-Aurora-Lakewood, CO (Metropolitan Stati...",8059,"Jefferson, CO"
427,24540,"Greeley, CO (Metropolitan Statistical Area)",8123,"Weld, CO"
451,25500,"Harrisonburg, VA (Metropolitan Statistical Area)",51947,"Rockingham + Harrisonburg, VA"
537,27980,"Kahului-Wailuku-Lahaina, HI (Metropolitan Stat...",15901,"Maui + Kalawao, HI"
562,28700,"Kingsport-Bristol, TN-VA (Metropolitan Statist...",51953,"Washington + Bristol, VA"


In [98]:
import requests
import json


api_key = "ED8C2AB7-DD09-4EE0-AC91-116A9E05348A"


years = '1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021'

url = f"https://apps.bea.gov/api/data/?UserID={api_key}&method=GetData&datasetname=Regional&TableName=CAINC30&LineCode=100&Year={years}&GeoFips=MSA&ResultFormat=json"
response = requests.get(url)
assert response.status_code == 200

# 100 stands for population

data = response.json()


data_series = data['BEAAPI']['Results']['Data']

# Create a DataFrame
df = pd.DataFrame(data_series)

In [99]:
df

,Code,GeoFips,GeoName,TimePeriod,CL_UNIT,UNIT_MULT,DataValue,NoteRef
0,CAINC30-100,00998,United States (Metropolitan Portion),1997,Number of persons,0,229081203,3
1,CAINC30-100,00998,United States (Metropolitan Portion),2005,Number of persons,0,250646689,3
2,CAINC30-100,00998,United States (Metropolitan Portion),2020,Number of persons,0,286183497,3
3,CAINC30-100,00998,United States (Metropolitan Portion),2018,Number of persons,0,283180665,3
4,CAINC30-100,00998,United States (Metropolitan Portion),2017,Number of persons,0,281262885,3
...,...,...,...,...,...,...,...,...
11160,CAINC30-100,49740,"Yuma, AZ (Metropolitan Statistical Area) *",2003,Number of persons,0,168003,3
11161,CAINC30-100,49740,"Yuma, AZ (Metropolitan Statistical Area) *",2014,Number of persons,0,198670,3
11162,CAINC30-100,49740,"Yuma, AZ (Metropolitan Statistical Area) *",1993,Number of persons,0,124892,3
11163,CAINC30-100,49740,"Yuma, AZ (Metropolitan Statistical Area) *",2004,Number of persons,0,172824,3


In [97]:
df['TimePeriod'].nunique()

385 * 29

11165

In [ ]:
https://apps.bea.gov/api/data/?UserID={api_key}&method=GetParameterValuesFiltered&datasetname=Regional&TargetParameter=LineCode&TableName=SAEMP25N&ResultFormat=json

In [81]:
import requests
import json


api_key = "ED8C2AB7-DD09-4EE0-AC91-116A9E05348A"

url = f"https://apps.bea.gov/api/data/?UserID={api_key}&method=GetParameterValuesFiltered&datasetname=Regional&TargetParameter=LineCode&TableName=CAINC30&ResultFormat=json"
response = requests.get(url)
assert response.status_code == 200



data = response.json()


# data_series = data['BEAAPI']['Results']['Data']

# # Create a DataFrame
# df = pd.DataFrame(data_series)

data

{'BEAAPI': {'Request': {'RequestParam': [{'ParameterName': 'USERID',
     'ParameterValue': 'ED8C2AB7-DD09-4EE0-AC91-116A9E05348A'},
    {'ParameterName': 'METHOD',
     'ParameterValue': 'GETPARAMETERVALUESFILTERED'},
    {'ParameterName': 'DATASETNAME', 'ParameterValue': 'REGIONAL'},
    {'ParameterName': 'TARGETPARAMETER', 'ParameterValue': 'LINECODE'},
    {'ParameterName': 'TABLENAME', 'ParameterValue': 'CAINC30'},
    {'ParameterName': 'RESULTFORMAT', 'ParameterValue': 'JSON'}]},
  'Results': {'ParamValue': [{'Key': '10',
     'Desc': '[CAINC30] Personal income'},
    {'Key': '100', 'Desc': '[CAINC30] Population'},
    {'Key': '110', 'Desc': '[CAINC30] Per capita personal income'},
    {'Key': '120', 'Desc': '[CAINC30] Per capita net earnings'},
    {'Key': '130',
     'Desc': '[CAINC30] Per capita personal current transfer receipts'},
    {'Key': '140', 'Desc': '[CAINC30] Per capita income maintenance benefits'},
    {'Key': '150',
     'Desc': '[CAINC30] Per capita unemployment